In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

Reusing TensorBoard on port 6006 (pid 44093), started 0:05:09 ago. (Use '!kill 44093' to kill it.)

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
                transforms.RandomVerticalFlip(),  
                transforms.RandomRotation(10),
                transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05)
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 256
p1 = .25

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base * 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base)
        self.batchnorm2 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm3 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm4 = nn.BatchNorm2d(channels_base)
        self.batchnorm5 = nn.BatchNorm2d(2)
        
    def forward(self, input):
        # encoder
        x = F.leaky_relu(self.batchnorm1(self.conv1(input)), negative_slope=0.1)
        x = y = F.dropout(x, p=p1)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        x = y1 = F.dropout(x, p=p1)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        
        # decoder
        x = F.leaky_relu(self.batchnorm3(self.convtrans1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm4(self.convtrans2(x + y1)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm5(self.convtrans3(x + y)), negative_slope=0.1)

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 256, 32, 32]           2,560
       BatchNorm2d-2          [-1, 256, 32, 32]             512
            Conv2d-3          [-1, 512, 16, 16]       1,180,160
       BatchNorm2d-4          [-1, 512, 16, 16]           1,024
   ConvTranspose2d-5            [-1, 512, 8, 8]       2,359,808
       BatchNorm2d-6            [-1, 512, 8, 8]           1,024
   ConvTranspose2d-7          [-1, 256, 16, 16]       1,179,904
       BatchNorm2d-8          [-1, 256, 16, 16]             512
   ConvTranspose2d-9            [-1, 2, 32, 32]           4,610
      BatchNorm2d-10            [-1, 2, 32, 32]               4
Total params: 4,730,118
Trainable params: 4,730,118
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 7.53
Params size (MB): 18.04
Estima

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00311555 (0.02102951), PSNR 25.06464767 (20.71741309), SSIM 0.77091473 (0.59015387)
Finished training epoch 0
Validate: MSE 0.00315097 (0.00268755), PSNR 25.01555061 (25.77875572), SSIM 0.70232701 (0.77341881)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00342871 (0.00265296), PSNR 24.64868927 (25.78847012), SSIM 0.75157017 (0.76666086)
Finished training epoch 1
Validate: MSE 0.00293955 (0.00261404), PSNR 25.31719398 (25.89807101), SSIM 0.70528316 (0.77199705)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00321791 (0.00259438), PSNR 24.92426682 (25.88291009), SSIM 0.74421138 (0.76710373)
Finished training epoch 2
Validate: MSE 0.00291712 (0.00257617), PSNR 25.35046196 (25.95620920), SSIM 0.70922524 (0.77173888)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00215445 (0.00255441), PSNR 26.66663551 (25.94863219), SSIM 0.77608907 (0.76844636)
Finished training epoch 3
Validate: MSE 0.00296066 (0.0

Epoch: 30, MSE 0.00223079 (0.00233283), PSNR 26.51542091 (26.34081350), SSIM 0.76843566 (0.77790885)
Finished training epoch 30
Validate: MSE 0.00280520 (0.00269035), PSNR 25.52035332 (25.77218169), SSIM 0.71907413 (0.77679481)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00269784 (0.00233771), PSNR 25.68984222 (26.33713244), SSIM 0.76532441 (0.77785997)
Finished training epoch 31
Validate: MSE 0.00252967 (0.00231756), PSNR 25.96936226 (26.40670952), SSIM 0.72478461 (0.78569558)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00244253 (0.00234284), PSNR 26.12159157 (26.32778500), SSIM 0.79541123 (0.77755739)
Finished training epoch 32
Validate: MSE 0.00251093 (0.00259923), PSNR 26.00165176 (25.92911583), SSIM 0.72213984 (0.77439217)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00217361 (0.00232672), PSNR 26.62817383 (26.35699368), SSIM 0.78235596 (0.77772669)
Finished training epoch 33
Validate: MSE 0.00260700 (0.00231718), PSNR 

Epoch: 60, MSE 0.00213014 (0.00226358), PSNR 26.71591949 (26.47521343), SSIM 0.79245394 (0.77889460)
Finished training epoch 60
Validate: MSE 0.00274690 (0.00241160), PSNR 25.61157608 (26.24425301), SSIM 0.72254437 (0.78260803)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.00224242 (0.00226770), PSNR 26.49283791 (26.46830939), SSIM 0.77125371 (0.77876713)
Finished training epoch 61
Validate: MSE 0.00258088 (0.00222520), PSNR 25.88231850 (26.58508161), SSIM 0.72345519 (0.78446765)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00236949 (0.00225950), PSNR 26.25345039 (26.48325167), SSIM 0.75202560 (0.77896492)
Finished training epoch 62
Validate: MSE 0.00260851 (0.00224555), PSNR 25.83606720 (26.54374176), SSIM 0.72528231 (0.78515536)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00175031 (0.00224856), PSNR 27.56884956 (26.50654635), SSIM 0.79210877 (0.77920528)
Finished training epoch 63
Validate: MSE 0.00241656 (0.00230875), PSNR 

Epoch: 90, MSE 0.00211984 (0.00221775), PSNR 26.73697090 (26.56341813), SSIM 0.78606480 (0.77912668)
Finished training epoch 90
Validate: MSE 0.00264914 (0.00231739), PSNR 25.76895714 (26.41319276), SSIM 0.72222447 (0.78619318)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.00233733 (0.00221244), PSNR 26.31280327 (26.57625451), SSIM 0.78637129 (0.77951079)
Finished training epoch 91
Validate: MSE 0.00248058 (0.00221643), PSNR 26.05447388 (26.59862979), SSIM 0.72820789 (0.78187986)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00179135 (0.00222494), PSNR 27.46820259 (26.55111520), SSIM 0.79202777 (0.77921182)
Finished training epoch 92
Validate: MSE 0.00251082 (0.00223376), PSNR 26.00184059 (26.56900495), SSIM 0.72823107 (0.78471195)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00178418 (0.00221856), PSNR 27.48560524 (26.56265239), SSIM 0.79165173 (0.77881166)
Finished training epoch 93
Validate: MSE 0.00287154 (0.00237959), PSNR 

Epoch: 120, MSE 0.00242001 (0.00220320), PSNR 26.16182899 (26.59324141), SSIM 0.78307736 (0.77853185)
Finished training epoch 120
Validate: MSE 0.00260500 (0.00230331), PSNR 25.84192467 (26.43604464), SSIM 0.72428656 (0.77792094)
Finished validation.
Starting training epoch 121
Epoch: 121, MSE 0.00223361 (0.00219488), PSNR 26.50991631 (26.60985531), SSIM 0.77747452 (0.77879992)
Finished training epoch 121
Validate: MSE 0.00275137 (0.00240582), PSNR 25.60450745 (26.25326230), SSIM 0.72723264 (0.78026000)
Finished validation.
Starting training epoch 122
Epoch: 122, MSE 0.00189785 (0.00219428), PSNR 27.21737862 (26.60899229), SSIM 0.77599180 (0.77872902)
Finished training epoch 122
Validate: MSE 0.00267622 (0.00222542), PSNR 25.72477150 (26.58761093), SSIM 0.72210407 (0.78452531)
Finished validation.
Starting training epoch 123
Epoch: 123, MSE 0.00217161 (0.00219496), PSNR 26.63219070 (26.61005081), SSIM 0.77933288 (0.77871514)
Finished training epoch 123
Validate: MSE 0.00263996 (0.00223

Validate: MSE 0.00250593 (0.00218739), PSNR 26.01030350 (26.66076832), SSIM 0.72276175 (0.78575160)
Finished validation.
Starting training epoch 150
Epoch: 150, MSE 0.00164087 (0.00218170), PSNR 27.84925652 (26.63561121), SSIM 0.79863971 (0.77827624)
Finished training epoch 150
Validate: MSE 0.00253011 (0.00222684), PSNR 25.96860504 (26.58244178), SSIM 0.72824061 (0.78133125)
Finished validation.
Starting training epoch 151
Epoch: 151, MSE 0.00240451 (0.00216430), PSNR 26.18973541 (26.66992647), SSIM 0.78303659 (0.77850345)
Finished training epoch 151
Validate: MSE 0.00261386 (0.00223268), PSNR 25.82718277 (26.57469178), SSIM 0.72049934 (0.78346523)
Finished validation.
Starting training epoch 152
Epoch: 152, MSE 0.00204441 (0.00217747), PSNR 26.89430809 (26.64274023), SSIM 0.78167307 (0.77822251)
Finished training epoch 152
Validate: MSE 0.00260419 (0.00221997), PSNR 25.84326363 (26.59640930), SSIM 0.71824026 (0.77873461)
Finished validation.
Starting training epoch 153
Epoch: 153, MS

Epoch: 179, MSE 0.00216916 (0.00216375), PSNR 26.63709068 (26.67298857), SSIM 0.77875030 (0.77789026)
Finished training epoch 179
Validate: MSE 0.00251902 (0.00219575), PSNR 25.98768616 (26.64150525), SSIM 0.71875715 (0.77911634)
Finished validation.
Starting training epoch 180
Epoch: 180, MSE 0.00220278 (0.00215885), PSNR 26.57029533 (26.67873243), SSIM 0.78427327 (0.77783914)
Finished training epoch 180
Validate: MSE 0.00256063 (0.00223126), PSNR 25.91653252 (26.56857942), SSIM 0.72397768 (0.78073830)
Finished validation.
Starting training epoch 181
Epoch: 181, MSE 0.00189557 (0.00214553), PSNR 27.22260857 (26.70734381), SSIM 0.79229599 (0.77785767)
Finished training epoch 181
Validate: MSE 0.00268209 (0.00219446), PSNR 25.71527290 (26.64774910), SSIM 0.72457278 (0.78488631)
Finished validation.
Starting training epoch 182
Epoch: 182, MSE 0.00217989 (0.00215434), PSNR 26.61565399 (26.68919924), SSIM 0.77313411 (0.77775525)
Finished training epoch 182
Validate: MSE 0.00267665 (0.00229

Validate: MSE 0.00254613 (0.00224913), PSNR 25.94119453 (26.53787168), SSIM 0.72769260 (0.78253309)
Finished validation.
Starting training epoch 209
Epoch: 209, MSE 0.00264713 (0.00214334), PSNR 25.77224541 (26.70988165), SSIM 0.76748621 (0.77712392)
Finished training epoch 209
Validate: MSE 0.00271142 (0.00230160), PSNR 25.66803169 (26.44336458), SSIM 0.71850085 (0.77872861)
Finished validation.
Starting training epoch 210
Epoch: 210, MSE 0.00210828 (0.00213007), PSNR 26.76072311 (26.73906463), SSIM 0.77301401 (0.77770194)
Finished training epoch 210
Validate: MSE 0.00266406 (0.00218401), PSNR 25.74456787 (26.66620625), SSIM 0.72021377 (0.78175508)
Finished validation.
Starting training epoch 211
Epoch: 211, MSE 0.00238128 (0.00213967), PSNR 26.23188972 (26.71712991), SSIM 0.75009871 (0.77744656)
Finished training epoch 211
Validate: MSE 0.00256674 (0.00243016), PSNR 25.90618324 (26.21462389), SSIM 0.71571231 (0.76673389)
Finished validation.
Starting training epoch 212
Epoch: 212, MS

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00253320 (0.00235979), PSNR 25.96331024 (26.32402389), SSIM 0.71603096 (0.77400245)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()